In [1]:
import re

In [12]:
text = "ソフトバンク, 現在の 支援活動 製品の お知らせ TOPICS トピックス HOT WORD MENU メニュー インターネット・固定電話 でんき ロボット 法人のお客さま1,Google Pixel | スマートフォン・携帯電話 | ソフトバンク, Google Pixel Previous Next Google Pixel 4a (5G) Google Pixel 5 Google Pixel 4a Google Pixel Buds マカロニえんぴつの音楽×トークライブイベント YouTube Premium Google のプレミアム会員サービス Google Pixel 一覧 Google Pixel を彩る豊富なアクセサリー お見積もり・ご購入 購入サポート に関するよくあるご質問 （別ウィンドウで開きます） PDF （モーダルで開きます"

In [38]:
d.check('hello')

True

In [108]:
import enchant
d = enchant.DictWithPWL("en","correction_vocab.txt")


In [26]:
collector = []
for items in text.split():
    if bool(re.compile('[A-Za-z]').match(items)):
        collector.append(items)
    elif bool(re.compile('.*[A-Za-z].*').match(items)):
        clean = re.sub(r'[^A-Za-z]','', items)
        if len(clean) > 1:
            collector.append(clean)

In [98]:
collector = list(set([x.lower() for x in collector]))

In [106]:
collector

['topics',
 'youtube',
 'previous',
 'buds',
 'word',
 'premium',
 'pdf',
 'google',
 'hot',
 'pixel',
 'next',
 'menu']

In [107]:
with open('correction_vocab.txt', 'w') as out:
    out.writelines("%s\n" % vocab for vocab in collector)

In [144]:
parsed_word = ['ソフトバンク', '現在', '支援', '活動', '製品', 'お知らせ', 'topics', 'トピックス', 'HO', 't', 'WORD', 'm', 'ENU', 'メニュー', 'インターネット', '固定電話', 'でんき', 'ロボット', '法人', 'お客様', 'Google Pixel', 'smartphone', '携帯電話', 'ソフトバンク', 'Google Pixel', 'pr', 'eviou', 's', 'NeXT', 'Google Pixel', 'a', 'g', 'Google Pixel', 'Google Pixel', 'a', 'Google Pixel', 'buds', 'マカロニえんぴつ', '音楽', 'トーク', 'ライブイベント', 'YouTube Premium', 'Google', 'プレミアム', '会員', 'サービス', 'Google Pixel', '一覧', 'Google Pixel', '彩る', '豊富', 'アクセサリー', 'お', '見積', 'ご', '購入', '購入', 'サポート', 'ある', 'ご', '質問', '別', 'ウィンドウ', '開く', 'pd', 'f', 'モーダル', '開く']

In [145]:
from collections import defaultdict
correction = defaultdict(list)
index = 0
for items in parsed_word:
    if len(items) < 3 and bool(re.compile('[A-Za-z]').match(items)):
        print(items,'=', index)
        
        # x + x+1
        if d.check(str(parsed_word[index] + parsed_word[index+1]).lower()):
            correction[str(parsed_word[index] + parsed_word[index+1]).lower()].append([index, index+1]) 
        # x-1 + x
        elif d.check(str(parsed_word[index-1] + parsed_word[index]).lower()):
            correction[str(parsed_word[index-1] + parsed_word[index]).lower()].append([index-1, index])
        # x-2 + x-1 + x
        elif d.check(str(parsed_word[index-2] + parsed_word[index-1] + parsed_word[index]).lower()):
            correction[str(parsed_word[index-2] + parsed_word[index-1] + parsed_word[index]).lower()].append([index-2, index-1, index])
        # x-1 + x + x+1
        elif d.check(str(parsed_word[index-1] + parsed_word[index] + parsed_word[index+1]).lower()):
            correction[str(parsed_word[index-1] + parsed_word[index]).lower()].append([index-1, index, index+1])
        # x + x+1 + x+2
        elif d.check(str(parsed_word[index] + parsed_word[index+1] + parsed_word[index+2]).lower()):
            correction[str(parsed_word[index] + parsed_word[index+1] + parsed_word[index+2]).lower()].append([index, index+1, index+2])

    index += 1

HO = 8
t = 9
m = 11
pr = 25
s = 27
a = 30
g = 31
a = 34
pd = 64
f = 65


In [146]:
correction

defaultdict(list,
            {'hot': [[8, 9], [8, 9]],
             'menu': [[11, 12]],
             'previous': [[25, 26, 27], [25, 26, 27]],
             'pdf': [[64, 65], [64, 65]]})

In [109]:
d.check('pdf')

True

In [147]:
for key, val in correction.items():
    flag = 1
    for items in val[0]:
        if flag:
            parsed_word[items] = key
            flag = 0
        else:
            parsed_word[items] = ''

In [148]:
[x for x in parsed_word if x is not '']

['ソフトバンク',
 '現在',
 '支援',
 '活動',
 '製品',
 'お知らせ',
 'topics',
 'トピックス',
 'hot',
 'WORD',
 'menu',
 'メニュー',
 'インターネット',
 '固定電話',
 'でんき',
 'ロボット',
 '法人',
 'お客様',
 'Google Pixel',
 'smartphone',
 '携帯電話',
 'ソフトバンク',
 'Google Pixel',
 'previous',
 'NeXT',
 'Google Pixel',
 'a',
 'g',
 'Google Pixel',
 'Google Pixel',
 'a',
 'Google Pixel',
 'buds',
 'マカロニえんぴつ',
 '音楽',
 'トーク',
 'ライブイベント',
 'YouTube Premium',
 'Google',
 'プレミアム',
 '会員',
 'サービス',
 'Google Pixel',
 '一覧',
 'Google Pixel',
 '彩る',
 '豊富',
 'アクセサリー',
 'お',
 '見積',
 'ご',
 '購入',
 '購入',
 'サポート',
 'ある',
 'ご',
 '質問',
 '別',
 'ウィンドウ',
 '開く',
 'pdf',
 'モーダル',
 '開く']